# BUILD LLM Bootcamp Day 2

### Install Libraries

In [1]:
!pip install snowflake-ml-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.3 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 26.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 26.6 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.3.0
    Uninstalling cachetools-5.3.0:
      Successfully uninstalled cachetools-5.3.0
  Attempting uninstall: xgboost
    Found existing installation: xgboost 1.7.1
    Uninstalling xgboost-1.7.1:
      Successfully uninstalled xgboost-1.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.4.0 requires cupy-cuda11x<12.0.0a0,>=9.5.0, which is not installed.


In [2]:
!pip install peft transformers==4.34.0 tokenizers vllm==0.2.1.post1 bitsandbytes datasets absl-py==1.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 89.8 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 174.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.6/28.6 MB 109.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 48.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 145.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 125.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 188.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.0/777.0 kB 156.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 174.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 3.1 MB/s eta 0:00:0000:0

### Import Libraries

In [16]:
from datasets import Dataset, ClassLabel
from transformers import AutoTokenizer, AutoModelForCausalLM
import sys
from utils import Concatenator
import pandas as pd
pd.set_option('display.max_colwidth', None)

import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
import os
import json
from transformers import TrainerCallback
from contextlib import nullcontext
from transformers import default_data_collator, Trainer, TrainingArguments

from snowflake.snowpark.session import Session
from snowflake.snowpark import VERSION
import snowflake.snowpark.functions as F
from snowflake.ml.registry import model_registry
from snowflake.ml.model import deploy_platforms
from snowflake.ml.model.models import llm

import logging 
logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)
logger = logging.getLogger("snowflake.ml")
logger.setLevel(logging.ERROR)

### Load Base Model and Tokenizer

In [5]:
# TODO: Set your Hugging Face token
!huggingface-cli login --token 

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
model_id="meta-llama/Llama-2-7b-chat-hf"
print('loading tokenizer')
tokenizer = LlamaTokenizer.from_pretrained(model_id)
print('loading model')
model = LlamaForCausalLM.from_pretrained(model_id, load_in_8bit=True, device_map='auto', torch_dtype=torch.float16)

loading tokenizer
loading model


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Prepare Datasets
### Load Datasets

*IMP NOTE: Please do NOT not run the following cell during the bootcamp session. Fine-tuning on the entire dataset could take over an hour.*

In [3]:
# def prepare_stratified_dataset(path, seed = 42):
#     raw_df = pd.read_json(path, lines=True)
#     raw_df['id'] = raw_df.index
#     ds = Dataset.from_pandas(raw_df, split='train')
#     cl = ClassLabel(num_classes=4, names=["EN", "FR", "DE", "ES"])
#     new_features = ds.features.copy()
#     new_features['lang_label'] = cl
#     cl_d = {l : cl.str2int(l) for l in ["EN", "FR", "DE", "ES"]}
#     def convert_lang(sample):
#         sample['lang_label'] = cl_d[sample['language']]
#         return sample
#     ds = ds.map(convert_lang, features=new_features)
#     ds_split = ds.train_test_split(test_size=0.15, stratify_by_column='lang_label', seed=42)
#     test_ds_split = ds_split['test'].train_test_split(test_size=2/3, stratify_by_column='lang_label', seed=42)
#     return ds_split['train'].to_pandas(), test_ds_split['train'].to_pandas(), test_ds_split['test'].to_pandas()

# train_df, eval_df, test_df = prepare_stratified_dataset(
#     'transcripts.json'
# )

Map:   0%|          | 0/1368 [00:00<?, ? examples/s]

In [ ]:
df = pd.read_json("transcripts.json", lines=True)
print(f"Total records: {df.shape[0]}")
train_df = df.head(100)
print(f"Train        : {train_df.shape[0]}")
eval_df = df[200:300]
print(f"Eval         : {eval_df.shape[0]}")
test_df = df.tail(100)
print(f"Test         : {test_df.shape[0]}")

In [4]:
train_df.head()

,instruction,input,output,language,id,lang_label
0,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""frosty: Greetings! I'm Frosty, your voice bot assistant for the holiday season. What's your name?\ncaller: Hey Frosty, my name is James.\nfrosty: Hi James! Where are you calling from?\ncaller: I'm calling from Sydney.\nfrosty: Nice to meet you, James from Sydney! Let's get started with your holiday wish list. What toy would you like to choose?\ncaller: Hmm... I'm not sure yet. What do you recommend?\nfrosty: I'm happy to help you explore our catalog! Does your child like plush toys?\ncaller: Yes, my little one definitely loves plush toys.\nfrosty: We have a sweet 2023 Holiday Fox 12-Inch Plush, or how about the Sesame Street Monster Meditation Elmo? Both are soft and cuddly.\ncaller: Oh, Elmo for sure!\nfrosty: The Sesame Street Monster Meditation Elmo is an excellent choice. Tell me, what do you enjoy the most about this time of the year?\ncaller: I love seeing all the decorations and spending time with my family.\nfrosty: That's wonderful! Do you have any favorite holiday memories?\ncaller: One year, we had a fantastic snowball fight in the park with our friends. It was so much fun!\nfrosty: That sounds like a terrific memory! So, are we adding Sesame Street Monster Meditation Elmo to your wish list?\ncaller: Yes! And you know what? Add the fox plushie as well.\nfrosty: You got it, adding the 2023 Holiday Fox 12-Inch Plush and Sesame Street Monster Meditation Elmo to your wish list. Have a great holiday season, James!\ncaller: Thanks, Frosty! You too!""","{""toy_list"": [""2023 Holiday Fox 12-Inch Plush"", ""Sesame Street Monster Meditation Elmo""], ""location"": ""Sydney""}",EN,245,0
1,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""caller: Hi there!\nfrosty: Hello! Welcome to our holiday hotline, I'm Frosty. How can I assist you today?\ncaller: Hey, Frosty. My name is Mike, and I live in Los Angeles.\nfrosty: Hi, Mike from Los Angeles! What can I help you with?\ncaller: I want to submit my holiday wish list, but I'm not sure what toys my son would like.\nfrosty: No problem! I'd be happy to help you explore our catalog. What are some of his interests?\ncaller: He loves playing with new gadgets and is a fan of Star Wars.\nfrosty: How about Star Wars LOLA animatronic droid? It's a fun gadget for any Star Wars fan.\ncaller: That sounds interesting. What else do you have?\nfrosty: Another option could be a Dog-E, the robot dog. It's a popular gadget for kids this holiday season.\ncaller: Let's go with the lola droid. But I also need a second toy.\nfrosty: Alright! Can you tell me more about what he enjoys during this time of the year?\ncaller: He loves playing outdoors and making snow forts with his friends.\nfrosty: In that case, he might be interested in the Tiny Land kids fort building kit. It allows him to create forts both indoors and outdoors, perfect for all seasons.\ncaller: Great, add both the lola droid and fort building kit to our wish list, please.\nfrosty: Excellent choices, Mike! I'll add the Star Wars LOLA animatronic droid and the Tiny Land kids fort building kit to your wish list. Have a fantastic holiday season!\n""","{""toy_list"": [""Star Wars LOLA animatronic droid"", ""Tiny Land kids fort building kit""], ""location"": ""Los Angeles""}",EN,968,0
2,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""\ncaller: Hello, I'm trying to submit a holiday wish list for my child but I'm not sure where to start.\nfrosty: Hello and welcome! I'd be happy to help. May I have your name please, and where are you calling from?\ncaller: My name is Laura, and I'm from Birmingham, Engla

In [5]:
datasets = {
    'train': Dataset.from_pandas(train_df),
    'eval': Dataset.from_pandas(eval_df),
    'test': Dataset.from_pandas(test_df)
}

### Apply Prompt to Datasets

In [6]:
train_prompt = f"""
[INST] <<SYS>>
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
<</SYS>>
### Instruction:
{{instruction}}
### Input:
{{input_}}
### Output:
{{output}}
{{eos_token}}
"""

eval_prompt = f"""
[INST] <<SYS>>
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
<</SYS>>
### Instruction:
{{instruction}}
### Input:
{{input_}}
### Output:
"""

In [7]:
def apply_train_template(sample):
    return {
        "text": train_prompt.format(
            instruction=sample["instruction"],
            input_=sample["input"].replace('\\n', '\n'),
            output=sample["output"],
            eos_token=tokenizer.eos_token,
        )
    }

def apply_eval_template(sample):
    return {
        "text": eval_prompt.format(
            instruction=sample["instruction"],
            input_=sample["input"].replace('\\n', '\n')
        )
    }



#applying template

datasets['train'] = datasets['train'].map(apply_train_template, remove_columns=list(datasets['train'].features))
for k in ['eval', 'test']:
    datasets[k] = datasets[k].map(apply_eval_template, remove_columns=list(datasets[k].features))


Map:   0%|          | 0/1162 [00:00<?, ? examples/s]

Map:   0%|          | 0/68 [00:00<?, ? examples/s]

Map:   0%|          | 0/138 [00:00<?, ? examples/s]

### Tokenize Datasets

In [8]:
for k, v in datasets.items():
    datasets[k] = v.map(
        lambda sample: tokenizer(sample["text"]),
        batched=True,
        
        remove_columns=list(v.features),
    ).map(Concatenator(), batched=True)

Map:   0%|          | 0/1162 [00:00<?, ? examples/s]

Map:   0%|          | 0/1162 [00:00<?, ? examples/s]

Map:   0%|          | 0/68 [00:00<?, ? examples/s]

Map:   0%|          | 0/68 [00:00<?, ? examples/s]

Map:   0%|          | 0/138 [00:00<?, ? examples/s]

Map:   0%|          | 0/138 [00:00<?, ? examples/s]

## Fine-tune Llama 2 Model


Fine-tuning is one form of model training. We start training from a pre-trained model and adjust a set of model parameters to better solve for a concrete task based on task specific data. Today we are going to fine-tune 7B Llama 2 model using LoRA (Low-Rank Adaptation)--which is a parameter efficient way of fine-tuning LLM. 

Instead of adjusting all the ~7B parameters, LoRA allows us to adjust only a percent of model weights--which can save compute and memory resources dramatically. For this lab, we will fine-tune our model using LoRA on a single A10 GPU. This will demostrate how good the inference can be on fine-tuned models even with limited compute.

### Configuration

We're passing `train_dataset` and `eval_dataset` that are used to generate loss calculation during fine-tuning process and we've set `output_weights_dir` as the directory where the fine-tuned weights will be stored after fine-tuning job completes.

To achieve good performance for the task, you will need at least 1 `num_epochs`, feel free to explore this on your own.

In [9]:
#setting the model into training mode
model.train()

#setting up training
def create_peft_config(model):
    from peft import (
        get_peft_model,
        LoraConfig,
        TaskType,
        prepare_model_for_kbit_training,
    )

    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8,
        lora_alpha=32,
        lora_dropout=0.05,
        target_modules = ["q_proj", "v_proj"]
    )

    # prepare int-8 model for training
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model, peft_config

# create peft config
model, lora_config = create_peft_config(model)


trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


In [36]:
# !rm -r output_weights_dir # deletes prior fine tuning weights
!mkdir output_weights_dir

In [10]:
output_dir = "output_weights_dir"
enable_profiler = False

config = {
    'lora_config': lora_config,
    'learning_rate': 1e-4,
    'num_train_epochs': 1,
    'gradient_accumulation_steps': 2,
    'per_device_train_batch_size': 2,
    'gradient_checkpointing': False,
}

# Define training args
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    bf16=True,  # Use BF16 if available
    # logging strategies
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="no",
    optim="adamw_torch_fused",
    max_steps=total_steps if enable_profiler else -1,
    **{k:v for k,v in config.items() if k != 'lora_config'}
)

### Finetuning

In [11]:
profiler = nullcontext() 

with profiler:
    # Create Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=datasets['train'],
        eval_dataset=datasets['eval'],
        data_collator=default_data_collator,
        callbacks=[profiler_callback] if enable_profiler else [],
    )

    # Start training
    trainer.train()
    
model.save_pretrained(output_dir)

Detected kernel version 5.4.181, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/envs/rapids/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/envs/rapids/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,1.377200
20,1.094700
30,0.993100
40,0.950500
50,0.890900
60,0.840400
70,0.800200
80,0.791000


## Log and Deploy Fine-tuned Llama 2

- Logging and deploying fine-tuned model on this setup (compute and other resources) will take about ~15mins

### Establish Secure Connection

*NOTE: Update [connection.json](../connection.json) and set your password, Hugging Face token, and replace '####' with your user number.*

In [12]:
# Create Snowflake Session object
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('select current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

User                        : DASH
Role                        : "DASH_SPCS"
Database                    : "DASH_DB"
Schema                      : "DASH_SCHEMA"
Warehouse                   : "DASH_L"
Snowflake version           : 7.41.0
Snowpark for Python version : 1.9.0


### Registery

In [13]:
MODEL_NAME = "LLAMA2_7b_CHAT"
MODEL_VERSION = "FineTunedV1.1"
DEPLOYMENT_NAME = "FINETUNED_LLAMA2"
MODEL_REGISTRY_DB = connection_parameters['database']
MODEL_REGISTRY_SCHEMA = connection_parameters['schema']
COMPUTE_POOL = connection_parameters['compute_pool']

registry = model_registry.ModelRegistry(
    session=session, 
    database_name=MODEL_REGISTRY_DB, 
    schema_name=MODEL_REGISTRY_SCHEMA, 
    create_if_not_exists=True)

create_model_registry() is in private preview since 0.2.0. Do not use it in production. 


In [14]:
registry.list_deployments(model_name=MODEL_NAME,model_version=MODEL_VERSION).to_pandas()

,MODEL_NAME,MODEL_VERSION,DEPLOYMENT_NAME,CREATION_TIME,TARGET_METHOD,TARGET_PLATFORM,SIGNATURE,OPTIONS,STAGE_PATH,ROLE
0,LLAMA2_7b_CHAT,FineTunedV1.1,FINETUNED_LLAMA2,2023-11-16 09:10:16.636000-08:00,infer,SNOWPARK_CONTAINER_SERVICES,"{\n ""inputs"": [\n {\n ""name"": ""input"",\n ""type"": ""STRING""\n }\n ],\n ""outputs"": [\n {\n ""name"": ""generated_text"",\n ""type"": ""STRING""\n }\n ]\n}","{\n ""compute_pool"": ""DASH_BUILD_GPU3"",\n ""num_gpus"": 1\n}",@DASH_DB.DASH_SCHEMA._SYSTEM_REGISTRY_DEPLOYMENTS_STAGE/FINETUNED_LLAMA2/,"""DASH_SPCS"""


In [18]:
# registry.delete_deployment(model_name=MODEL_NAME,model_version=MODEL_VERSION,deployment_name=DEPLOYMENT_NAME)

In [19]:
# registry.delete_model(model_name=MODEL_NAME,model_version=MODEL_VERSION,delete_artifact=True)

### Reference Llama 2 from Fine Tuning

In [20]:
%%time
#referencing our fine tuned model
#referencing huggingface token
options = llm.LLMOptions(
    token=connection_parameters['huggingface_token'],
    max_batch_size=100,
)
#referencing our fine tuned weights and using the hugging face token to merge with base llama model
llama_model = llm.LLM(
    model_id_or_path='output_weights_dir',
    options=options
)

# log model in registry
llama_model_ref = registry.log_model(
    model_name=MODEL_NAME,
    model_version=MODEL_VERSION,
    model=llama_model
)

# deploy model
llama_model_ref.deploy(
    deployment_name=DEPLOYMENT_NAME, 
    platform=deploy_platforms.TargetPlatform.SNOWPARK_CONTAINER_SERVICES,
    permanent=True, 
   options={"compute_pool": COMPUTE_POOL, "num_gpus": 1})

/opt/conda/envs/rapids/lib/python3.8/site-packages/snowflake/ml/model/_packager/model_env/model_env.py:353: UserWarning: Found dependencies specified as pip requirements. This may prevent model deploying to Snowflake Warehouse.
  warnings.warn(


CPU times: user 1.74 s, sys: 292 ms, total: 2.03 s
Wall time: 2min 56s


{'name': 'DASH_DB.DASH_SCHEMA.FINETUNED_LLAMA2',
 'platform': <TargetPlatform.SNOWPARK_CONTAINER_SERVICES: 'SNOWPARK_CONTAINER_SERVICES'>,
 'target_method': 'infer',
 'signature': ModelSignature(
                     inputs=[
                         FeatureSpec(dtype=DataType.STRING, name='input')
                     ],
                     outputs=[
                         FeatureSpec(dtype=DataType.STRING, name='generated_text')
                     ]
                 ),
 'options': {'compute_pool': 'DASH_BUILD_GPU3', 'num_gpus': 1},
 'details': {'image_name': 'sfsenorthamerica-build-spcs.registry.snowflakecomputing.com/dash_db/dash_schema/snowml_repo/a2dc5950178b8bf1c3764f03dddcbe86ea99c226:latest',
  'service_spec': "spec:\n  container:\n  - env:\n      MODEL_ZIP_STAGE_PATH: /DASH_DB.DASH_SCHEMA.SNOWML_MODEL_97005C9C84C811EEB02EE246F4FD5A27/model.zip\n      NUM_WORKERS: 1\n      SNOWML_USE_GPU: true\n      TARGET_METHOD: infer\n      _CONCURRENT_REQUESTS_MAX: 1\n    image: sfsen

In [21]:
# if its already deployed run the following:
llama_model_ref = model_registry.ModelReference(
    registry=registry, 
    model_name=MODEL_NAME, 
    model_version=MODEL_VERSION)

## Inference on Eval Dataset using fine-tuned Llama 2

In [22]:
eval_df['input'] = eval_df.apply(
    lambda x: eval_prompt.format(
        instruction=x["instruction"],
        input_=x["input"].replace('\\n', '\n')
    ), axis=1
)
eval_df.reset_index(drop=True, inplace=True)
eval_df.head()

,instruction,input,output,language,id,lang_label
0,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"\n[INST] <<SYS>>\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n<</SYS>>\n### Instruction:\nExtract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.\n### Input:\n""caller: Hi there! \nfrosty: Hello! Welcome to our Holiday Wish List line! How can I help you today? \ncaller: My name is Sarah and I'm calling from London. I wanted to submit a wish list for my child. \nfrosty: Wonderful, Sarah! It's great to have you on the line. What country do you live in? \ncaller: I live in the United Kingdom. \nfrosty: That's fantastic! What's on your wish list? \ncaller: I think my child would like the transformers bumblebee. \nfrosty: Great choice! Why are you interested in this toy? \ncaller: My child loves action figures and Transformers. I think he'll have a lot of fun with this one. \nfrosty: That sounds perfect! How do you and your family plan to celebrate the holiday season? \ncaller: We usually have a big family gathering at our house with lots of food and games. \nfrosty: That sounds lovely! Do you have a favorite holiday memory or pastime? \ncaller: My favorite memory is baking holiday cookies with my mom when I was a child. \nfrosty: That's a beautiful memory! So, just to confirm, you would like to add the Transformers Rise of the Beasts Beast-Mode Bumblebee to your wish list? \ncaller: Yes, that's correct! \nfrosty: Excellent! Thank you, Sarah, and I hope you and your family have a wonderful holiday season! \ncaller: Thank you! Have a great day! \nfrosty: You too! Goodbye!""\n### Output:\n","{""toy_list"": [""Transformers Rise of the Beasts Beast-Mode Bumblebee""], ""location"": ""London""}",EN,39,0
1,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"\n[INST] <<SYS>>\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n<</SYS>>\n### Instruction:\nExtract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.\n### Input:\n""caller: Hello, my name is Shawn, and I'm calling from Vancouver. I need to submit a holiday wish list for my kids but I could use some help picking out the perfect toys.\nfrosty: Hi Shawn, from Vancouver! Let's find the best toys for your kids. Can you tell me a bit about their interests?\ncaller: My daughter is 5 years old, and she loves Sesame Street, especially Elmo. As for my son, he's 9 and really into Pok\u00e9mon.\nfrosty: I have just the toys in mind! How about the Sesame Street Monster Meditation Elmo for your daughter and the Pok\u00e9mon 8-inch Plush First Partner Three-Pack for your son?\ncaller: Those sound perfect! Please add them to the wish list.\nfrosty: I've added the Sesame Street Monster Meditation Elmo for your daughter and the Pok\u00e9mon 8-inch Plush First Partner Three-Pack for your son to the wish list. I hope they enjoy their gifts, and I wish your family a fantastic holiday season in Vancouver!""\n### Output:\n","{""toy_list"": [""Sesame Street Monster Meditation Elmo"", ""Pok\u00e9mon 8-Inch Plush First Partner Three-Pack""], ""location"": ""Vancouver""}",EN,525,0
2,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"\n[INST] <<SYS>>\nBelow is an instruction that describes a task, paired with an input that provides further context. W

In [23]:
eval_df['predicted'] = llama_model_ref.predict(deployment_name=DEPLOYMENT_NAME,data=eval_df)#.head()
eval_df[['output', 'predicted']].head()

,output,predicted
0,"{""toy_list"": [""Transformers Rise of the Beasts Beast-Mode Bumblebee""], ""location"": ""London""}","{\n""location"": ""London"",\n""toy_list"": [""Transformers Rise of the Beasts Beast-Mode Bumblebee""]\n}\n"
1,"{""toy_list"": [""Sesame Street Monster Meditation Elmo"", ""Pok\u00e9mon 8-Inch Plush First Partner Three-Pack""], ""location"": ""Vancouver""}","{""location"": ""Vancouver"", ""toy_list"": [""Sesame Street Monster Meditation Elmo"", ""Pok\u00e9mon 8-inch Plush First Partner Three-Pack""]}\n"
2,"{""toy_list"": [""Marvel's Spidey and His Amazing Friends Web Spinners playset""], ""location"": ""Melbourne""}","{\n""location"": ""Melbourne"",\n""toy_list"": [""Marvel's Spidey and His Amazing Friends Web Spinners playset""]\n}\n"
3,"{""toy_list"": [""LeapFrog Magic Adventures Microscope""], ""location"": ""Chicago""}","{\n""location"": ""Chicago"",\n""toy_list"": [""LeapFrog Magic Adventures Microscope""]\n}\n"
4,"{""toy_list"": [""Furby interactive plush toy""], ""location"": ""Wellington""}","{\n""location"": ""Wellington"",\n""toy_list"": [""furby""]\n}\n"


## Clean Up Resources

Delete deployment and the model

In [ ]:
llama_model_ref.delete_deployment(deployment_name=DEPLOYMENT_NAME)
llama_model_ref.delete_model(delete_artifact=True)